## Day 13: Distress Signal

[![nbviewer](https://raw.githubusercontent.com/jupyter/design/master/logos/Badges/nbviewer_badge.svg)](https://nbviewer.org/github/mazharenko/AoC-2022/tree/HEAD/notebooks/day13/puzzle.ipynb)

In [1]:
#!value --name sampleRaw 
[1,1,3,1,1]
[1,1,5,1,1]

[[1],[2,3,4]]
[[1],4]

[9]
[[8,7,6]]

[[4,4],4,4]
[[4,4],4,4,4]

[7,7,7,7]
[7,7,7]

[]
[3]

[[[]]]
[[]]

[1,[2,[3,[4,[5,6,7]]]],8,9]
[1,[2,[3,[4,[5,6,0]]]],8,9]

In [2]:
#!value --name actualRaw --from-file ./data_actual.txt

### Desiging the type

Each list consists of integers or other lists, or both. This can be represented as a recursive DU.

Comparison is a built-in concept and we want to respect it. The default comparison implementation does not meet our requirements though, but it can be overriden.

Luckily, the default comparison for lists does meet our requirements.

In [3]:
[<CustomComparison; CustomEquality>]
type Packet = 
    | Int of int
    | Packets of Packet list
    interface IComparable<Packet> with
        member this.CompareTo other = 
            match this,other with
            | Int i, Int j -> compare i j
            | (Packets p1, Packets p2) -> 
                compare p1 p2
            | Int i, Packets p ->
                compare (Packets [Int i]) (Packets p)
            | Packets p, Int i -> 
                compare (Packets p) (Packets [Int i]) 
    override this.Equals other =
        match other with
        | :? Packet as p -> compare this p = 0
        | _ -> false
    interface IComparable with
        member this.CompareTo other = 
            match other with 
            | :? Packet as p -> (this :> IComparable<Packet>).CompareTo(p)
            | _ -> -1

### Parsing

In [4]:
#r "nuget:Farkle, 6.3.2"
open Farkle
open Farkle.Builder

Installed Packages Farkle, 6.3.2

In [5]:
#load "../common/common.fsx"

let private number = Terminals.int "Int"
let private packet = nonterminal "Packet"
packet.SetProductions(
    !@ number => (fun i -> Int i),
    !& "[" .>>. (sepBy (Terminal.Literal ",") packet) .>> "]" => Packets
)

let private parser = RuntimeFarkle.build packet

let parsePacket s = 
    RuntimeFarkle.parseString parser s
    |> Result.get

let parsePackets raw = 
    Pattern2.read id raw 
    |> Array.map (fun x -> 
        let [|p1; p2|] = Pattern1.read parsePacket x
        p1,p2
    )



In [6]:
#!share sampleRaw --from value
#!share actualRaw --from value
let samplePackets = parsePackets sampleRaw
let actualPackets = parsePackets actualRaw
samplePackets

index Item1 Item2 0 Item FSharpList<Packet>
 - Item: 1
 - Item: 1
 - Item: 3
 - Item: 1
 - Item: 1 Item FSharpList<Packet>
 - Item: 1
 - Item: 1
 - Item: 5
 - Item: 1
 - Item: 1 1 Item FSharpList<Packet>
 - Item: FSharpList<Packet>
 - Item: 1
 - Item: FSharpList<Packet>
 - Item: 2
 - Item: 3
 - Item: 4 Item FSharpList<Packet>
 - Item: FSharpList<Packet>
 - Item: 1
 - Item: 4 2 Item FSharpList<Packet>
 - Item: 9 Item FSharpList<Packet>
 - Item: FSharpList<Packet>
 - Item: 8
 - Item: 7
 - Item: 6 3 Item FSharpList<Packet>
 - Item: FSharpList<Packet>
 - Item: 4
 - Item: 4
 - Item: 4
 - Item: 4 Item FSharpList<Packet>
 - Item: FSharpList<Packet>
 - Item: 4
 - Item: 4
 - Item: 4
 - Item: 4
 - Item: 4 4 Item FSharpList<Packet>
 - Item: 7
 - Item: 7
 - Item: 7
 - Item: 7 Item FSharpList<Packet>
 - Item: 7
 - Item: 7
 - Item: 7 5 Item FSharpList<Packet>
 Item FSharpList<Packet>
 - Item: 3 6 Item FSharpList<Packet>
 - Item: FSharpList<Packet>
 - Item: FSharpList<Packet>
 Item FSharpList<Packet>
 - Item: FSharpList<Packet>
 7 Item FSharpList<Packet>
 - Item: 1
 - Item: FSharpList<Packet>
 - Item: 2
 - Item: FSharpList<Packet>
 - Item: 3
 - Item: FSharpList<Packet>
Int 4
Packets [Int 5; Int 6; Int 7]
 - Item: 8
 - Item: 9 Item FSharpList<Packet>
 - Item: 1
 - Item: FSharpList<Packet>
 - Item: 2
 - Item: FSharpList<Packet>
 - Item: 3
 - Item: FSharpList<Packet>
Int 4
Packets [Int 5; Int 6; Int 0]
 - Item: 8
 - Item: 9

### Part 1

In [14]:
let inOrderSum packets = 
    samplePackets
    |> Array.indexed
    |> Array.filter (fun (i, (p1,p2)) -> p2 > p1)
    |> Array.map (fst >> ((+)1))
    |> Array.sum

inOrderSum samplePackets

13

In [13]:
inOrderSum actualPackets

5529

### Part 2

In [18]:
let div1 = parsePacket "[[2]]"
let div2 = parsePacket "[[6]]"

let dividerIndices packets = 
    let sorted = 
        packets
        |> Array.collect (fun (p1, p2)-> [|p1;p2|])
        |> Array.append [|div1; div2|]
        |> Array.sort
    (Array.findIndex ((=)div1) sorted + 1) * (Array.findIndex ((=)div2) sorted + 1)
    
dividerIndices samplePackets

140

In [19]:
dividerIndices actualPackets

27690